In [240]:
# -*- coding: utf-8 -*-

"""
Created May 7, 2022
"""

import glob
import os
import pickle

import numpy as np
import pandas as pd
from IPython.display import display_html
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

import utils


BASED_STRUCTURED_DIR = "/dartfs/rc/nosnapshots/H/HillB-scratch/OAI/structured_data"
PREDICTIONS_CSV = "/dartfs/rc/nosnapshots/H/HillB-scratch/OAI/model_predictions/predictions_2022_12_06_08_09_19_298378.csv"
DICOM_METADATA_PATH = "./data/dicom_metadata_df.pkl"
VARIABLES_OF_INTEREST = [
    "(0008, 0070) Manufacturer",
    "(0008, 1090) Manufacturer's Model Name",
    "(0012, 0030) Clinical Trial Site ID",
    "(0018, 1000) Device Serial Number",
    "Site ID - Model",
    "hospital_site",
]
CLINICAL_WAVES_TO_FOLLOWUP = {
    "00": "Screening Visit",
    "01": "12 month Annual Visit",
    "03": "24 month Annual Visit",
    "05": "36 month Annual Visit",
    "06": "48 month Annual Visit",
    "07": "60 month Annual Visit",
    "08": "72 month Annual Visit",
    "09": "84 month Annual Visit",
    "10": "96 month Annual Visit",
    "11": "108 month Annual Visit",
}


## Structured Data

Here we combine the structured data from multiple timepoints.

In [241]:
original_dataframes: dict[str, pd.DataFrame] = {}

clinical_data_files = glob.glob(os.path.join(BASED_STRUCTURED_DIR, "AllClinical*.txt"))
xray_files = glob.glob(os.path.join(BASED_STRUCTURED_DIR, "XRay*.txt"))
enrollee_files = glob.glob(os.path.join(BASED_STRUCTURED_DIR, "Enrollees*.txt"))

combined_files = sorted(clinical_data_files + xray_files)

for filename in combined_files:
    name = os.path.basename(filename).replace(".txt", "").lower()

    df = utils.read_data_frame(os.path.join(BASED_STRUCTURED_DIR, filename))

    if df is None:
        continue

    original_dataframes[name] = df

if not len(enrollee_files):
    raise ValueError("missing Enrollee.txt from structured data")

df_site = utils.read_data_frame(enrollee_files[0])[["id", "v00site"]].astype("string")


In [242]:
dfs = {}

for side in ["left", "right"]:
    subset_cols = (
        ["id", "kooskpl", "womkpl"] if side == "left" else ["id", "kooskpr", "womkpr"]
    )

    dfs[side] = utils.concatenate_from_timepoints(
        original_dataframes,
        dataset_substring="allclinical",
        subset=subset_cols,
    )

    dfs[side] = dfs[side].merge(df_site, on="id", how="inner")
    dfs[side].rename(columns={"v00site": "hospital_site"}, inplace=True)

    utils.validate_column(
        pd.Series(dfs[side].columns),
        subset_cols + ["visit", "hospital_site"],
    )

    dfs[side].columns = [
        "id",
        "koos_pain_subscore",
        "womac_pain_subscore",
        "visit",
        "hospital_site",
    ]  # type:ignore
    dfs[side]["side"] = side

all_knee_pain_scores = pd.concat([val for val in dfs.values()])
all_knee_pain_scores["visit"] = all_knee_pain_scores["visit"].apply(
    lambda x: CLINICAL_WAVES_TO_FOLLOWUP.get(x, pd.NA)
)
all_knee_pain_scores


,id,koos_pain_subscore,womac_pain_subscore,visit,hospital_site,side
0,9000099,100.0,0.0,Screening Visit,B,left
1,9000099,88.9,0.0,12 month Annual Visit,B,left
2,9000099,80.6,4.0,24 month Annual Visit,B,left
3,9000099,97.2,0.0,<NA>,B,left
4,9000099,83.3,2.0,36 month Annual Visit,B,left
...,...,...,...,...,...,...
48737,9999878,94.4,0.0,60 month Annual Visit,C,right
48738,9999878,97.2,0.0,72 month Annual Visit,C,right
48739,9999878,97.2,0.0,84 month Annual Visit,C,right
48740,9999878,84.4,1.0,96 month Annual Visit,C,right


In [244]:
with open(DICOM_METADATA_PATH, "rb") as f:
    dicom_metadata = pickle.load(f)

dicom_metadata["join_id"] = dicom_metadata.index.values.astype("str")
dicom_metadata["dicom_join_id"] = dicom_metadata["(0010, 0020) Patient ID"].astype(
    "str"
)
dicom_metadata["Site ID - Model"] = dicom_metadata[
    "(0012, 0030) Clinical Trial Site ID"
].str.cat(dicom_metadata["(0008, 1090) Manufacturer's Model Name"])

dicom_metadata[
    ["join_id", "dicom_join_id"]
    + [v for v in VARIABLES_OF_INTEREST if v != "hospital_site"]
]


,join_id,dicom_join_id,"(0008, 0070) Manufacturer","(0008, 1090) Manufacturer's Model Name","(0012, 0030) Clinical Trial Site ID","(0018, 1000) Device Serial Number",Site ID - Model
00700804,00700804,9659701,AGFA,ADC_51xx,58,1134,58ADC_51xx
04168501,04168501,9485404,"""GE Healthcare""","""Definium 5000""",46,NaN,"46""Definium 5000"""
02127103,02127103,9596662,"FUJI PHOTO FILM Co., ltd.",NaN,23,NaN,NaN
03558001,03558001,9018489,Swissray,ddR Formula System,34,S402607,34ddR Formula System
01597503,01597503,9953183,LS100,Lumisys,10,NaN,10Lumisys
...,...,...,...,...,...,...,...
03844601,03844601,9257071,FUJIFILM Corporation,NaN,58,NaN,NaN
00507901,00507901,9101951,Agfa-Gevaert AG,ADC_5146,46,2205,46ADC_5146
02256603,02256603,9297051,NaN,NaN,11,NaN,NaN
03625601,03625601,9836547,FUJIFILM Corporation,NaN,10,NaN,NaN


## Merging Dicom Metadata

We can join the OAI structured data with the dicom metadata by joining on the patient ID.  Our variables of interest for grouping koos predictions are:

- by manufacturer
- by model
- by serial number
- by site-id
- by site-id / model

In [245]:
df_combined = dicom_metadata.merge(
    all_knee_pain_scores,
    left_on="dicom_join_id",
    right_on="id",
    how="left",
).dropna(subset=["id", "koos_pain_subscore"])

dicom_timepoint = "(0012, 0051) Clinical Trial Time Point Description"

df_combined["bin_koos"] = df_combined["koos_pain_subscore"].apply(utils.binarize_koos)
df_combined["koos_pain_subscore"].astype("string")
df_combined[dicom_timepoint] = df_combined[dicom_timepoint].astype("string")
df_combined = df_combined.dropna(subset=["bin_koos"])
df_combined = df_combined.loc[df_combined["visit"] == df_combined[dicom_timepoint]]
df_combined


,"(0008, 0005) Specific Character Set","(0008, 0008) Image Type","(0008, 0012) Instance Creation Date","(0008, 0013) Instance Creation Time","(0008, 0016) SOP Class UID","(0008, 0018) SOP Instance UID","(0008, 0020) Study Date","(0008, 0022) Acquisition Date","(0008, 0030) Study Time","(0008, 0032) Acquisition Time",...,join_id,dicom_join_id,Site ID - Model,id,koos_pain_subscore,womac_pain_subscore,visit,hospital_site,side,bin_koos
1,ISO_IR 100,"[DERIVED, PRIMARY]",20050513,091344,1.2.840.10008.5.1.4.1.1.1,1.3.6.1.4.1.21767.172.16.8.165.1158825987.343....,20050513,20050513,092624,092624,...,00700804,9659701,58ADC_51xx,9659701,80.6,2.0,12 month Annual Visit,D,left,1.0
11,ISO_IR 100,"[DERIVED, PRIMARY]",20050513,091344,1.2.840.10008.5.1.4.1.1.1,1.3.6.1.4.1.21767.172.16.8.165.1158825987.343....,20050513,20050513,092624,092624,...,00700804,9659701,58ADC_51xx,9659701,100.0,0.0,12 month Annual Visit,D,right,0.0
28,ISO_IR 100,"[ORIGINAL, PRIMARY]",NaN,NaN,1.2.840.10008.5.1.4.1.1.1.1,1.3.6.1.4.1.21767.127.0.0.1.1377304274.0.0.1.1,20130806,20130806,142822,143521,...,04168501,9485404,"46""Definium 5000""",9485404,94.4,1.0,96 month Annual Visit,C,left,0.0
38,ISO_IR 100,"[ORIGINAL, PRIMARY]",NaN,NaN,1.2.840.10008.5.1.4.1.1.1.1,1.3.6.1.4.1.21767.127.0.0.1.1377304274.0.0.1.1,20130806,20130806,142822,143521,...,04168501,9485404,"46""Definium 5000""",9485404,94.4,1.0,96 month Annual Visit,C,right,0.0
42,NaN,"[DERIVED, PRIMARY, POST_PROCESSED, , , , , , 1...",NaN,NaN,1.2.840.10008.5.1.4.1.1.1,1.3.6.1.4.1.21767.172.16.8.165.1183056852.849....,20070611,20070611,112952.000,113258.531,...,02127103,9596662,NaN,9596662,90.6,1.0,24 month Annual Visit,E,left,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539975,NaN,NaN,NaN,NaN,1.2.840.10008.5.1.4.1.1.7,1.3.6.1.4.1.21767.172.16.10.155.1188580343.595...,20070822,20070822,000000,000000.0,...,02256603,9297051,NaN,9297051,86.1,1.0,36 month Annual Visit,B,right,1.0
539988,NaN,DERIVED,NaN,NaN,1.2.840.10008.5.1.4.1.1.1,1.3.6.1.4.1.21767.127.0.0.1.1300739971.0.0.1.1,20110308,20110308,091253,091432.500,...,03625601,9836547,NaN,9836547,75.0,2.0,72 month Annual Visit,B,left,1.0
539998,NaN,DERIVED,NaN,NaN,1.2.840.10008.5.1.4.1.1.1,1.3.6.1.4.1.21767.127.0.0.1.1300739971.0.0.1.1,20110308,20110308,091253,091432.500,...,03625601,9836547,NaN,9836547,80.6,2.0,72 month Annual Visit,B,right,1.0
540002,NaN,"[ORIGINAL, SECONDARY, RADIOGRAPH]",20050929,233900,1.2.840.10008.5.1.4.1.1.1,1.3.6.1.4.1.21767.172.16.8.165.1143501729.639....,20040615,20040615,000000,000000.0,...,00096003,9444196,22Lumisys,9444196,44.4,10.0,Screening Visit,E,left,1.0


We can aggregate koos scores by `Manufacterer model`.

## Koos Aggregation

Aggregation of `binarized koos` allows us to see the percentage of severe knee pain x-rays for each manufacturer.

In [247]:
koos_variables = ["koos_pain_subscore", "bin_koos"]

html_raw_tables = []
for target_variable in VARIABLES_OF_INTEREST:
    df_agg = (
        df_combined[koos_variables + [target_variable]]
        .replace("", np.nan)
        .dropna(subset=[target_variable])
        .groupby(target_variable)
        .agg(["mean", "median", "count"])
    )

    html_raw_tables.append(
        df_agg.style.set_table_attributes("style='display:inline'")
        .set_caption(target_variable)
        ._repr_html_()
    )

display_html("".join(html_raw_tables), raw=True)


(0008, 0070) Manufacturer 
 
 
   
 koos_pain_subscore 
 bin_koos 
 
 
   
 mean 
 median 
 count 
 mean 
 median 
 count 
 
 
 (0008, 0070) Manufacturer 
   
   
   
   
   
   
 
 
 
 
 "GE Healthcare" 
 88.483013 
 94.400000 
 7759 
 0.336126 
 0.000000 
 7759 
 
 
 AGFA 
 87.368271 
 94.400000 
 9455 
 0.364252 
 0.000000 
 9455 
 
 
 Agfa-Gevaert AG 
 87.975816 
 94.400000 
 7869 
 0.360147 
 0.000000 
 7869 
 
 
 FUJI PHOTO FILM Co., ltd. 
 81.182679 
 88.900000 
 2396 
 0.481219 
 0.000000 
 2396 
 
 
 FUJIFILM Corporation 
 88.121656 
 94.400000 
 7402 
 0.341124 
 0.000000 
 7402 
 
 
 GE Healthcare 
 89.328483 
 97.200000 
 811 
 0.314427 
 0.000000 
 811 
 
 
 LS100 
 85.277453 
 91.700000 
 4271 
 0.401077 
 0.000000 
 4271 
 
 
 Philips Medical Systems 
 83.130526 
 91.150000 
 190 
 0.447368 
 0.000000 
 190 
 
 
 SIEMENS 
 81.545692 
 88.900000 
 650 
 0.467692 
 0.000000 
 650 
 
 
 Swissray 
 82.619614 
 88.900000 
 8713 
 0.463790 
 0.000000 
 8713 
 
 
 

 
 (0008, 1090) Manufacturer's Model Name 
 
 
   
 koos_pain_subscore 
 bin_koos 
 
 
   
 mean 
 median 
 count 
 mean 
 median 
 count 
 
 
 (0008, 1090) Manufacturer's Model Name 
   
   
   
   
   
   
 
 
 
 
 "Definium 5000" 
 88.325206 
 94.400000 
 6927 
 0.340840 
 0.000000 
 6927 
 
 
 "Thunder Platform" 
 89.796875 
 97.200000 
 832 
 0.296875 
 0.000000 
 832 
 
 
 ADC_5146 
 87.975816 
 94.400000 
 7869 
 0.360147 
 0.000000 
 7869 
 
 
 ADC_51xx 
 87.368271 
 94.400000 
 9455 
 0.364252 
 0.000000 
 9455 
 
 
 DigitalDiagnost 
 82.902151 
 90.600000 
 186 
 0.451613 
 0.000000 
 186 
 
 
 Discovery XR656 
 89.328483 
 97.200000 
 811 
 0.314427 
 0.000000 
 811 
 
 
 FLUOROSPOT_COMPACT 
 83.300000 
 83.300000 
 2 
 0.500000 
 0.500000 
 2 
 
 
 Lumisys 
 85.432595 
 91.700000 
 3930 
 0.400763 
 0.000000 
 3930 
 
 
 SIEMENS FD-X 
 81.540278 
 88.900000 
 648 
 0.467593 
 0.000000 
 648 
 
 
 ddR Combi System 
 77.600000 
 83.300000 
 8 
 0.500000 
 0.500000 
 8 
 
 
 ddR Formula System 
 82.855338 
 88.900000 
 3269 
 0.455797 
 0.000000 
 3269 
 
 
 ddR Modulaire System 
 82.549572 
 88.900000 
 5378 
 0.467274 
 0.000000 
 5378 
 
 
 ddR Multi System 
 76.520690 
 83.300000 
 58 
 0.586207 
 1.000000 
 58 
 
 
 digital DIAGNOST 
 93.750000 
 100.000000 
 4 
 0.250000 
 0.000000 
 4 
 
 
 

 
 (0012, 0030) Clinical Trial Site ID 
 
 
   
 koos_pain_subscore 
 bin_koos 
 
 
   
 mean 
 median 
 count 
 mean 
 median 
 count 
 
 
 (0012, 0030) Clinical Trial Site ID 
   
   
   
   
   
   
 
 
 
 
 1 
 97.050000 
 97.050000 
 2 
 0.000000 
 0.000000 
 2 
 
 
 5 
 88.900000 
 88.900000 
 2 
 0.500000 
 0.500000 
 2 
 
 
 10 
 90.197833 
 97.200000 
 2492 
 0.274077 
 0.000000 
 2492 
 
 
 11 
 86.789731 
 94.400000 
 8949 
 0.370544 
 0.000000 
 8949 
 
 
 22 
 84.840659 
 91.150000 
 182 
 0.428571 
 0.000000 
 182 
 
 
 23 
 80.585016 
 87.500000 
 4418 
 0.491852 
 0.000000 
 4418 
 
 
 33 
 72.200000 
 72.200000 
 2 
 1.000000 
 1.000000 
 2 
 
 
 34 
 83.183036 
 90.600000 
 560 
 0.442857 
 0.000000 
 560 
 
 
 35 
 82.775842 
 88.900000 
 7248 
 0.462748 
 0.000000 
 7248 
 
 
 38 
 95.850000 
 95.850000 
 2 
 0.000000 
 0.000000 
 2 
 
 
 40 
 83.350000 
 89.100000 
 6 
 0.500000 
 0.500000 
 6 
 
 
 41 
 98.600000 
 98.600000 
 2 
 0.000000 
 0.000000 
 2 
 
 
 46 
 88.232712 
 94.400000 
 15618 
 0.348124 
 0.000000 
 15618 
 
 
 55 
 76.400000 
 76.400000 
 2 
 0.500000 
 0.500000 
 2 
 
 
 58 
 87.698090 
 94.400000 
 13140 
 0.356393 
 0.000000 
 13140 
 
 
 I1 
 87.150000 
 97.200000 
 8 
 0.250000 
 0.000000 
 8 
 
 
 

 
 (0018, 1000) Device Serial Number 
 
 
   
 koos_pain_subscore 
 bin_koos 
 
 
   
 mean 
 median 
 count 
 mean 
 median 
 count 
 
 
 (0018, 1000) Device Serial Number 
   
   
   
   
   
   
 
 
 
 
 08.02.366 
 100.000000 
 100.000000 
 2 
 0.000000 
 0.000000 
 2 
 
 
 08.02.399 
 87.500000 
 87.500000 
 2 
 0.500000 
 0.500000 
 2 
 
 
 1003 
 83.300000 
 83.300000 
 2 
 0.500000 
 0.500000 
 2 
 
 
 

## Model predictions

These predictions are generated by the `koos` deep learning model.  They are written to the path indicated by `PREDICTIONS_CSV`.  Xrays were scored using a corresponding slurm model creating a `koos` numeric prediction and a binned prediction mapped to either `0` or `1`.

In [248]:
df_predictions = pd.read_csv(
    PREDICTIONS_CSV,
    names=["batch", "koos_prediction", "patient_id", "bin_koos_prediction"],
    header=None,
    skiprows=1,
)

df_predictions["join_id"] = df_predictions["patient_id"].map("{:08}".format)
df_predictions


,batch,koos_prediction,patient_id,bin_koos_prediction,join_id
0,0,97.930750,1652703,0.0,01652703
1,1,80.273544,2002706,1.0,02002706
2,2,91.560130,1623104,0.0,01623104
3,3,83.731750,4181102,1.0,04181102
4,4,86.088936,2300601,1.0,02300601
...,...,...,...,...,...
26515,3,82.593605,2063801,1.0,02063801
26516,4,98.483200,2338901,0.0,02338901
26517,5,94.040940,1371103,0.0,01371103
26518,6,82.667530,1632904,1.0,01632904


## Merging Predictions

Model predictions are merged with dicom metadata by joining again on patient ID.  This allows us to have both the actual and predicted koos value for a given xray.  The sides for each xray are included in the analysis.

In [249]:
koos_variables = ["koos_pain_subscore", "bin_koos"]

df_combined_predictions = df_combined.merge(
    df_predictions, left_on="join_id", right_on="join_id", how="inner"
).dropna(subset=["koos_prediction"] + VARIABLES_OF_INTEREST)

df_combined_predictions[
    ["join_id", "visit", "side"] + koos_variables + VARIABLES_OF_INTEREST
]


,join_id,visit,side,koos_pain_subscore,bin_koos,"(0008, 0070) Manufacturer","(0008, 1090) Manufacturer's Model Name","(0012, 0030) Clinical Trial Site ID","(0018, 1000) Device Serial Number",Site ID - Model,hospital_site
0,00700804,12 month Annual Visit,left,80.6,1.0,AGFA,ADC_51xx,58,1134,58ADC_51xx,D
1,00700804,12 month Annual Visit,right,100.0,0.0,AGFA,ADC_51xx,58,1134,58ADC_51xx,D
6,03558001,72 month Annual Visit,left,100.0,0.0,Swissray,ddR Formula System,34,S402607,34ddR Formula System,A
7,03558001,72 month Annual Visit,right,100.0,0.0,Swissray,ddR Formula System,34,S402607,34ddR Formula System,A
16,02102701,36 month Annual Visit,left,88.9,0.0,AGFA,ADC_51xx,58,1522,58ADC_51xx,D
...,...,...,...,...,...,...,...,...,...,...,...
52962,02143601,36 month Annual Visit,right,97.2,0.0,AGFA,ADC_51xx,58,1134,58ADC_51xx,D
52971,00333904,Screening Visit,left,100.0,0.0,AGFA,ADC_51xx,58,1134,58ADC_51xx,D
52972,00333904,Screening Visit,right,61.1,1.0,AGFA,ADC_51xx,58,1134,58ADC_51xx,D
52975,00507901,Screening Visit,left,100.0,0.0,Agfa-Gevaert AG,ADC_5146,46,2205,46ADC_5146,C


In [250]:
def prediction_performance(ytrue, ypred):
    stats = {}

    try:
        cm = confusion_matrix(ytrue, ypred)

        spec_sum = cm[1, 0] + cm[1, 1]
        sen_sum = cm[0, 0] + cm[0, 1]
        prec_sum = cm[0, 0] + cm[1, 0]

        if spec_sum == 0 or sen_sum == 0 or prec_sum == 0:
            raise Exception

        if not len(ypred) or len(cm) <= 1:
            raise Exception

        stats["Accuracy"] = (cm[0, 0] + cm[1, 1]) / cm.sum()
        stats["Precision"] = cm[0, 0] / (cm[0, 0] + cm[1, 0])
        stats["Specificity"] = cm[1, 1] / (spec_sum)
        stats["Sensitivity"] = cm[0, 0] / (sen_sum)
        stats["ER"] = (cm[0, 1] + cm[1, 0]) / cm.sum()
        stats["FPR"] = 1 - stats["Specificity"]
        stats["TP Count"] = cm[0, 0]
        stats["TN Count"] = cm[1, 1]
    except:
        stats["Accuracy"] = np.nan
        stats["Precision"] = np.nan
        stats["Specificity"] = np.nan
        stats["Sensitivity"] = np.nan
        stats["ER"] = np.nan
        stats["FPR"] = np.nan
        stats["TP Count"] = np.nan
        stats["TN Count"] = np.nan

    return stats


global_ytrue = df_combined_predictions["bin_koos"]
global_ypred = df_combined_predictions["bin_koos_prediction"]
global_cm = confusion_matrix(global_ytrue, global_ypred)

prediction_performance(global_ytrue, global_ypred)


{'Accuracy': 0.5068156930861307,
 'Precision': 0.5996302382908792,
 'Specificity': 0.4437785388127854,
 'Sensitivity': 0.5483063051781354,
 'ER': 0.4931843069138693,
 'FPR': 0.5562214611872146,
 'TP Count': 8757,
 'TN Count': 4665}

## Global Performance

The global performance of prediction gives an accuracy of 0.507 with a precision of 0.600.  Sensitivity is 0.548 and specificity is 0.444.

## Koos Accuracy by Manufacturer

Grouping the predictions by manufacturer shows varying levels of accuracy.  The most accurately predicted pain scores occured when the manufacturer model was the `SIEMENS`.  The least accurate occurred with `Agfa-Gevaert AG`.

## Accuracy Discrepancies

Many of the xray manufacturers show widely varying sensitivities and specificities.  Aside from the large drop in accuracy with `Agfa-Gevaert AG`, there is a large difference between sensitivity even at similar counts.  These differences also hold for model names and site id.

In [251]:
def grouping_func(row):
    ytrue = row["bin_koos"]
    ypred = row["bin_koos_prediction"]
    d = {"Count": len(ypred)}
    d.update(prediction_performance(ytrue, ypred))

    return pd.Series(d)


html_raw_tables = []
for grouping_var in VARIABLES_OF_INTEREST:
    df_combined_predictions[grouping_var] = df_combined_predictions[
        grouping_var
    ].astype("string")
    df_agg = (
        df_combined_predictions[df_combined_predictions[grouping_var] != ""]
        .groupby([grouping_var])
        .apply(grouping_func)
        .sort_values(by=["Count"], ascending=False)
    )

    html_raw_tables.append(
        df_agg.style.set_table_attributes("style='display:inline'")
        .set_caption(grouping_var)
        ._repr_html_()
    )

display_html("".join(html_raw_tables), raw=True)


,Count,Accuracy,Precision,Specificity,Sensitivity,ER,FPR,TP Count,TN Count
"(0008, 0070) Manufacturer",,,,,,,,,
AGFA,9343.000000,0.551857,0.647763,0.373187,0.653085,0.448143,0.626813,3895.000000,1261.000000
Swissray,8421.000000,0.525828,0.542532,0.290644,0.729874,0.474172,0.709356,3291.000000,1137.000000
Agfa-Gevaert AG,7867.000000,0.427482,0.637922,0.753884,0.243893,0.572518,0.246116,1228.000000,2135.000000
SIEMENS,650.000000,0.563077,0.564583,0.312500,0.783237,0.436923,0.687500,271.000000,95.000000
Philips Medical Systems,190.000000,0.552632,0.586207,0.435294,0.647619,0.447368,0.564706,68.000000,37.000000
GE Healthcare,12.000000,nan,nan,nan,nan,nan,nan,nan,nan
,Count,Accuracy,Precision,Specificity,Sensitivity,ER,FPR,TP Count,TN Count
"(0008, 1090) Manufacturer's Model Name",,,,,,,,,
ADC_51xx,9343.000000,0.551857,0.647763,0.373187,0.653085,0.448143,0.626813,3895.000000,1261.000000


## Feature Selection Analysis

In [252]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2, SelectKBest


TEST_VARIABLES = [
    "(0008, 0080) Institution Name",
    "(0008, 0008) Image Type",
    "(0008, 1030) Study Description",
    "(0018, 1000) Device Serial Number",
    "(0018, 1020) Software Versions",
    "(0018, 5101) View Position",
    *VARIABLES_OF_INTEREST,
]


def feature_select(dataset, yvar="bin_koos", xvars=[]):
    if not len(xvars):
        raise ValueError("xvars cannot be empty")

    le = LabelEncoder()
    oe = OrdinalEncoder()

    X = dataset[xvars].astype(str)
    y = dataset[yvar].values

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=0
    )

    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)

    oe.fit(X_train)
    X_train_enc = oe.transform(X_train)
    X_test_enc = oe.transform(X_test)

    fs = SelectKBest(score_func=chi2, k="all")
    fs.fit(X_train_enc, y_train_enc)

    cols = fs.get_support(indices=True)
    feature_names = X.iloc[:, cols].columns

    X_train_fs = fs.transform(X_train_enc)
    X_test_fs = fs.transform(X_test_enc)

    for i in range(len(fs.scores_)):
        print("Feature %s: %f" % (feature_names[i], fs.scores_[i]))


feature_select(df_combined_predictions, yvar="bin_koos", xvars=TEST_VARIABLES)


Feature (0008, 0080) Institution Name: 20.746257
Feature (0008, 0008) Image Type: 525.462432
Feature (0008, 1030) Study Description: 67.581534
Feature (0018, 1000) Device Serial Number: 34.454259
Feature (0018, 1020) Software Versions: 17.500046
Feature (0018, 5101) View Position: 0.422160
Feature (0008, 0070) Manufacturer: 391.664286
Feature (0008, 1090) Manufacturer's Model Name: 643.504002
Feature (0012, 0030) Clinical Trial Site ID: 161.297454
Feature (0018, 1000) Device Serial Number: 34.454259
Feature Site ID - Model: 197.672877
Feature hospital_site: 30.200640


In [253]:
pd.set_option("display.max_rows", 75)

target_variable = "hospital_site"
model_name = "(0008, 1090) Manufacturer's Model Name"

df_combined.groupby([target_variable, model_name])["id"].agg(["count"])


count
hospital_site (0008, 1090) Manufacturer's Model Name       
A                                                         0
              "Definium 5000"                             4
              "Thunder Platform"                          2
              ADC_5146                                    4
              ADC_51xx                                    8
              DigitalDiagnost                           128
              Discovery XR656                             0
              FLUOROSPOT_COMPACT                          0
              Lumisys                                     2
              SIEMENS FD-X                                0
              ddR Combi System                            8
              ddR Formula System                       2466
              ddR Modulaire System                     5378
              ddR Multi System                           58
              digital DIAGNOST                            2
B                                                       203
              "Definium 5000"                             0
              "Thunder Platform"                          0
              ADC_5146                                    0
              ADC_51xx                                    0
              DigitalDiagnost                             0
              Discovery XR656                             0
              FLUOROSPOT_COMPACT                          0
              Lumisys                                  3568
              SIEMENS FD-X                                0
              ddR Combi System                            0
              ddR Formula System                          0
              ddR Modulaire System                        0
              ddR Multi System                            0
              digital DIAGNOST                            0
C                                                         0
              "Definium 5000"                          6923
              "Thunder Platform"                        830
              ADC_5146                                 7865
              ADC_51xx                                    0
              DigitalDiagnost                             0
              Discovery XR656                             0
              FLUOROSPOT_COMPACT                          0
              Lumisys                                     0
              SIEMENS FD-X                                0
              ddR Combi System                            0
              ddR Formula System                          0
              ddR Modulaire System                        0
              ddR Multi System                            0
              digital DIAGNOST                            0
D                                                         0
              "Definium 5000"                             0
              "Thunder Platform"                          0
              ADC_5146                                    0
              ADC_51xx                                 9447
              DigitalDiagnost                             0
              Discovery XR656                           811
              FLUOROSPOT_COMPACT                          0
              Lumisys                                     0
              SIEMENS FD-X                                0
              ddR Combi System                            0
              ddR Formula System                          0
              ddR Modulaire System                        0
              ddR Multi System                            0
              digital DIAGNOST                            0
E                                                      1600
              "Definium 5000"                             0
              "Thunder Platform"                          0
              ADC_5146                                    0
              ADC_51xx                                    0
              DigitalDiagnost     